# Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import TimeSeriesSplit

# metrics
from sklearn.metrics import make_scorer, mean_absolute_error, median_absolute_error, mean_squared_error

# cross-validation
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict, GridSearchCV, RandomizedSearchCV

# preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

# regression models
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Load data

You can assume data is already clean.

In [2]:
# Read in the weather data csv
df_raw = pd.read_csv('data/sydney_weather.csv', parse_dates=['Date'])
df_raw.sort_values('Date', inplace=True)
df_raw

,Date,MinTemp,MaxTemp,MedTemp,Rainfall,Sunshine,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm
0,2008-02-01,19.5,22.4,20.95,15.6,0.0,92.0,84.0,1017.6,1017.4,20.7,20.9
1,2008-02-02,19.5,25.6,22.55,6.0,2.7,83.0,73.0,1017.9,1016.4,22.4,24.8
2,2008-02-03,21.6,24.5,23.05,6.6,0.1,88.0,86.0,1016.7,1015.6,23.5,23.0
3,2008-02-04,20.2,22.8,21.50,18.8,0.0,83.0,90.0,1014.2,1011.8,21.4,20.9
4,2008-02-05,19.7,25.7,22.70,77.4,0.0,88.0,74.0,1008.3,1004.8,22.5,25.5
...,...,...,...,...,...,...,...,...,...,...,...,...
3339,2017-06-21,8.6,19.6,14.10,0.0,7.8,73.0,52.0,1025.9,1025.3,10.5,17.9
3340,2017-06-22,9.3,19.2,14.25,0.0,9.2,78.0,53.0,1028.5,1024.6,11.0,18.7
3341,2017-06-23,9.4,17.7,13.55,0.0,2.7,85.0,56.0,1020.8,1015.0,10.2,17.3
3342,2017-06-24,10.1,19.3,14.70,0.0,9.3,56.0,35.0,1017.3,1015.1,12.4,19.0


# Feature engineering

For the sake of learning, we can keep this simple by just using "MedTemp". Use `def_med` dataframe for performing feature engineering and for training the models.

In [3]:
df = df_raw[['Date', 'MedTemp']].copy()

## Date and time features

Add date and time features (columns) such as year, month, day, season, day of the week, etc.

Example:

In [4]:
from datetime import date

In [5]:
def get_season(fecha):
    for Y in range(2008, 2018):
        estaciones = [
            (1, date(Y, 1, 1), date(Y, 3, 20)),
            (2, date(Y, 3, 21), date(Y, 6, 20)),
            (3, date(Y, 6, 21), date(Y, 9, 22)),
            (4, date(Y, 9, 23), date(Y, 12, 20)),
            (1, date(Y, 12, 21), date(Y, 12, 31))
        ]
        for estacion, inicio, fin in estaciones:
            if inicio <= fecha <= fin:
                return estacion

In [6]:
df['Day_of_week'] = df['Date'].dt.dayofweek
df['Season'] = pd.to_datetime(df['Date']).apply(get_season)
df.head()

C:\Users\Alumne_mati1\AppData\Local\Temp\ipykernel_3004\351075824.py:11: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if inicio <= fecha <= fin:


,Date,MedTemp,Day_of_week,Season
0,2008-02-01,20.95,4,1
1,2008-02-02,22.55,5,1
2,2008-02-03,23.05,6,1
3,2008-02-04,21.50,0,1
4,2008-02-05,22.70,1,1


## Lag features

Add columns that correspond to the values from the previous N days (one column for each previous day).

You can decide which N you think is better or train multiple models with different datasets with different values of N to see which one gives the best performance.

Use the `shift` function in pandas for moving the values down. Example:

In [7]:
df[f'MedTemp_lag_1'] = df['MedTemp'].shift(1)
df[f'MedTemp_lag_2'] = df['MedTemp'].shift(2)
df.head()

,Date,MedTemp,Day_of_week,Season,MedTemp_lag_1,MedTemp_lag_2
0,2008-02-01,20.95,4,1,NaN,NaN
1,2008-02-02,22.55,5,1,20.95,NaN
2,2008-02-03,23.05,6,1,22.55,20.95
3,2008-02-04,21.50,0,1,23.05,22.55
4,2008-02-05,22.70,1,1,21.50,23.05


## Rolling features

Calculate "summarized features" of the last N days (e.g., 3 days).

In [8]:
df['MedTemp_rolling_avg_3'] = df['MedTemp'].rolling(3, min_periods=3).mean()
df['MedTemp_expanding_avg'] = df['MedTemp'].expanding().mean()
df.head(5)

,Date,MedTemp,Day_of_week,Season,MedTemp_lag_1,MedTemp_lag_2,MedTemp_rolling_avg_3,MedTemp_expanding_avg
0,2008-02-01,20.95,4,1,NaN,NaN,NaN,20.950000
1,2008-02-02,22.55,5,1,20.95,NaN,NaN,21.750000
2,2008-02-03,23.05,6,1,22.55,20.95,22.183333,22.183333
3,2008-02-04,21.50,0,1,23.05,22.55,22.366667,22.012500
4,2008-02-05,22.70,1,1,21.50,23.05,22.416667,22.150000


## Target

Compute the target value we want to predict with a prediction window of 1 day.

Feel free to try other prediction windows, e.g. forecasting 2, 3 or 7 days. Further predictions should have worse results, as the current temperature values are less informative of the far future.

In [9]:
df['target'] = df['MedTemp'].shift(-1).values
df.tail()

,Date,MedTemp,Day_of_week,Season,MedTemp_lag_1,MedTemp_lag_2,MedTemp_rolling_avg_3,MedTemp_expanding_avg,target
3339,2017-06-21,14.10,2,3,15.65,14.75,14.833333,18.940284,14.25
3340,2017-06-22,14.25,3,3,14.10,15.65,14.666667,18.938881,13.55
3341,2017-06-23,13.55,4,3,14.25,14.10,13.966667,18.937268,14.70
3342,2017-06-24,14.70,5,3,13.55,14.25,14.166667,18.936001,13.45
3343,2017-06-25,13.45,6,3,14.70,13.55,13.900000,18.934360,NaN


## Clean NaNs

Drop the NaNs that have appeared during the feature engineering process.

In [10]:
df = df.dropna()
df.shape

(3341, 9)

# Cross-validation

In [11]:
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MedAE': make_scorer(median_absolute_error),
    'MSE': make_scorer(mean_squared_error)
}

In [20]:
def custom_cv_ts(df, models, X, y, cv):
    """
    Perform time series cross-validation with a given model, X, y, and cv object.
    """
    # Create empty train and test MAE lists
    train_maes = []
    val_maes = []
    train_mse = []
    val_mse= []
    train_medae = []
    val_medae = []
    best_pa = []

    # Cross-validation
    for i, (train_index, val_index) in enumerate(cv.split(X)):
        # Train and val data in this split
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_val, y_val = X.iloc[val_index], y.iloc[val_index]

        # Nested train and validation data from previous train data
        X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train, y_train, test_size=0.1,
                                                              random_state=42, shuffle=False)
        
        # Calculate validation metrics after training each model
        metrics = []
        for dtree in models:
            dtree.fit(X_train2, y_train2)
            y_pred_val2 = dtree.predict(X_val2)
            metrics.append( mean_absolute_error(y_val2, y_pred_val2) )
            
        # Select the best model based on validation MAE and retrain it with the whole training data
        best_model_idx = np.argmin(metrics)
        best_model = models[best_model_idx]
        best_model.fit(X_train, y_train)
        
        # Predict and compute MAEs of the train and val sets
        y_pred_train = best_model.predict(X_train)
        y_pred_val = best_model.predict(X_val)
        train_maes.append( mean_absolute_error(y_train, y_pred_train) )
        val_maes.append( mean_absolute_error(y_val, y_pred_val) )
        train_mse.append( mean_squared_error(y_train, y_pred_train) )
        val_mse.append( mean_squared_error(y_val, y_pred_val) )
        train_medae.append( median_absolute_error(y_train, y_pred_train) )
        val_medae.append( median_absolute_error(y_val, y_pred_val) )
        if isinstance(best_model, RandomizedSearchCV):        
            best_pa.append(best_model.best_params_)
                                    
        # Show info
        print(i)
        dates = df.loc[X.index, 'Date']
        print('Best validation model:', best_model_idx)
        print('Train dates:', dates.iloc[train_index].min().date(), '-', dates.iloc[train_index].max().date())
        print('Train MAE:', round(train_maes[-1], 2))
        print('Train MSE:', round(train_mse[-1], 2))
        print('Train MedAE:', round(train_medae[-1], 2))
        print('Validation dates:', dates.iloc[val_index].min().date(), '-', dates.iloc[val_index].max().date())
        print('Validation MAE:', round(val_maes[-1], 2))
        print('Validation MSE:', round(val_mse[-1], 2))
        print('Validation MedAE:', round(val_medae[-1], 2))
        print()
        if isinstance(best_model, RandomizedSearchCV):  
            print("Best Parameters: ",best_model.best_params_)

    return np.array(train_maes), np.array(val_maes), np.array(train_mse), np.array(val_mse), np.array(train_medae), np.array(val_medae), np.array(best_pa)

In [13]:
X = df.drop(['Date', 'target'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

## Example

In [85]:
df.shape

(3341, 9)

In [86]:
# Create TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=50, test_size=7)

# Models to try in each cross-validation split
models = [
    DecisionTreeRegressor(max_depth=2),
    DecisionTreeRegressor(max_depth=4),
    DecisionTreeRegressor(max_depth=6),
]

train_maes, valid_maes, train_mse, valid_mse, train_medae, valid_medae, best_pa = custom_cv_ts(df, models, X_train, y_train, cv=tscv)

Best validation model: 2
Train dates: 2008-02-03 - 2014-09-09
Train MAE: 1.32
Train MSE: 3.23
Train MedAE: 0.99
Validation dates: 2014-09-10 - 2014-09-16
Validation MAE: 1.28
Validation MSE: 2.33
Validation MedAE: 0.95

Best validation model: 2
Train dates: 2008-02-03 - 2014-09-16
Train MAE: 1.32
Train MSE: 3.23
Train MedAE: 0.99
Validation dates: 2014-09-17 - 2014-09-23
Validation MAE: 0.54
Validation MSE: 0.52
Validation MedAE: 0.33

Best validation model: 2
Train dates: 2008-02-03 - 2014-09-23
Train MAE: 1.32
Train MSE: 3.22
Train MedAE: 0.99
Validation dates: 2014-09-24 - 2014-09-30
Validation MAE: 2.47
Validation MSE: 8.45
Validation MedAE: 2.23

Best validation model: 2
Train dates: 2008-02-03 - 2014-09-30
Train MAE: 1.3
Train MSE: 3.12
Train MedAE: 0.98
Validation dates: 2014-10-01 - 2014-10-07
Validation MAE: 3.06
Validation MSE: 18.65
Validation MedAE: 1.99

Best validation model: 2
Train dates: 2008-02-03 - 2014-10-07
Train MAE: 1.3
Train MSE: 3.12
Train MedAE: 0.98
Validatio

In [87]:
print('Mean train MAE:', train_maes.mean().round(2), 'ºC')
print('Mean validation MAE:', valid_maes.mean().round(2), 'ºC')
print()
print('Mean train MSE:', train_mse.mean().round(2), 'ºC')
print('Mean validation MSE:', valid_mse.mean().round(2), 'ºC')
print()
print('Mean train MedAE:', train_medae.mean().round(2), 'ºC')
print('Mean validation MedAE:', valid_medae.mean().round(2), 'ºC')

Mean train MAE: 1.37 ºC
Mean validation MAE: 1.48 ºC

Mean train MSE: 3.43 ºC
Mean validation MSE: 4.06 ºC

Mean train MedAE: 1.03 ºC
Mean validation MedAE: 1.19 ºC


## Baseline

In [54]:
tscv = TimeSeriesSplit(n_splits=50, test_size=7)

# Models to try in each cross-validation split
models = [
    DummyRegressor(strategy='mean')    
]

dum_train_maes, dum_valid_maes, dum_train_mse, dum_valid_mse, dum_train_medae, dum_valid_medae, dum_best_pa = custom_cv_ts(df, models, X_train, y_train, cv=tscv)

Best validation model: 0
Train dates: 2008-02-03 - 2014-09-09
Train MAE: 3.53
Train MSE: 17.37
Train MedAE: 3.38
Validation dates: 2014-09-10 - 2014-09-16
Validation MAE: 1.41
Validation MSE: 3.2
Validation MedAE: 0.83

Best validation model: 0
Train dates: 2008-02-03 - 2014-09-16
Train MAE: 3.52
Train MSE: 17.32
Train MedAE: 3.38
Validation dates: 2014-09-17 - 2014-09-23
Validation MAE: 2.59
Validation MSE: 7.11
Validation MedAE: 2.52

Best validation model: 0
Train dates: 2008-02-03 - 2014-09-23
Train MAE: 3.52
Train MSE: 17.29
Train MedAE: 3.38
Validation dates: 2014-09-24 - 2014-09-30
Validation MAE: 2.38
Validation MSE: 12.35
Validation MedAE: 0.67

Best validation model: 0
Train dates: 2008-02-03 - 2014-09-30
Train MAE: 3.52
Train MSE: 17.28
Train MedAE: 3.38
Validation dates: 2014-10-01 - 2014-10-07
Validation MAE: 2.46
Validation MSE: 9.89
Validation MedAE: 1.62

Best validation model: 0
Train dates: 2008-02-03 - 2014-10-07
Train MAE: 3.51
Train MSE: 17.26
Train MedAE: 3.37
Val

In [55]:
print('Mean train MAE:', dum_train_maes.mean().round(2), 'ºC')
print('Mean validation MAE:', dum_valid_maes.mean().round(2), 'ºC')
print()
print('Mean train MSE:', dum_train_mse.mean().round(2), 'ºC')
print('Mean validation MSE:', dum_valid_mse.mean().round(2), 'ºC')
print()
print('Mean train MedAE:', dum_train_medae.mean().round(2), 'ºC')
print('Mean validation MedAE:', dum_valid_medae.mean().round(2), 'ºC')

Mean train MAE: 3.53 ºC
Mean validation MAE: 3.83 ºC

Mean train MSE: 17.38 ºC
Mean validation MSE: 19.77 ºC

Mean train MedAE: 3.38 ºC
Mean validation MedAE: 3.72 ºC


## Models

In [88]:
from keras.models import Sequential
from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasRegressor

In [67]:
def create_model():
    model = Sequential()
    model.add(X_train.shape[1])
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_absolute_error', optimizer='adam')
    return model

In [15]:
## No lo he acabado haciendo porque dura demasiado tiempo, tambien se le podria poner la red neuronal

# Create TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=25, test_size=7)

param_dist_knn = {
    "scale": [StandardScaler(), RobustScaler(), MinMaxScaler()],
    "knn__n_neighbors": [3, 5, 8, 10, 12, 15, 20],
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1, 2]
}

knn = Pipeline([
    ('scale', None),
    ('knn', KNeighborsRegressor(n_jobs=-1))
])

param_dist_rf = {
    "max_depth": [3, 5, 10, 20, 50, 100, None],
    "min_samples_split": np.arange(2, 30),
    "min_samples_leaf": np.arange(1, 11),
    "criterion": ["poisson", "friedman_mse", "squared_error", "absolute_error"],
    "n_estimators": np.arange(2,200),
    "max_features": ["sqrt", "log2", 0.2, 0.4, 0.6, 0.8],  
    "bootstrap": [True, False]
}

rf = RandomForestRegressor(n_jobs=-1)

param_dist_gb = {
    "max_depth": [3, 5, 10, 20, 50, 100, None],
    "min_samples_split": np.arange(2, 30),
    "min_samples_leaf": np.arange(1, 11),
    "criterion": ["friedman_mse", "squared_error"],
    "n_estimators": np.arange(2,200),
    "max_features": ["sqrt", "log2", 0.2, 0.4, 0.6, 0.8], 
    "learning_rate": [ 0.09, 0.091, 0.092, 0.093, 0.094]
}

gb = GradientBoostingRegressor()

# Models to try in each cross-validation split
models = [
    LinearRegression(n_jobs=-1),
    RandomizedSearchCV(knn, param_distributions=param_dist_knn,
                              return_train_score=True, cv=tscv,
                              n_iter= 50, n_jobs=-1),
    DecisionTreeRegressor(max_depth=6),
    RandomizedSearchCV(rf, param_distributions=param_dist_rf,
                             return_train_score=True, cv=tscv,
                             verbose=1, n_iter=50,
                             n_jobs=-1),
    RandomizedSearchCV(gb, param_distributions=param_dist_gb,
                             return_train_score=True,
                             cv=tscv, verbose=1, n_iter=50,
                             n_jobs=-1),
]

train_maes, valid_maes, train_mse, valid_mse, train_medae, valid_medae, best_pa = custom_cv_ts(df, models, X_train, y_train, cv=tscv)

Fitting 25 folds for each of 50 candidates, totalling 1250 fits
Fitting 25 folds for each of 50 candidates, totalling 1250 fits
Best validation model: 2
Train dates: 2008-02-03 - 2015-03-03
Train MAE: 1.31
Train MSE: 3.14
Train MedAE: 1.01
Validation dates: 2015-03-04 - 2015-03-10
Validation MAE: 1.58
Validation MSE: 2.74
Validation MedAE: 1.92

Fitting 25 folds for each of 50 candidates, totalling 1250 fits
Fitting 25 folds for each of 50 candidates, totalling 1250 fits
Best validation model: 2
Train dates: 2008-02-03 - 2015-03-10
Train MAE: 1.31
Train MSE: 3.14
Train MedAE: 1.01
Validation dates: 2015-03-11 - 2015-03-17
Validation MAE: 1.88
Validation MSE: 5.62
Validation MedAE: 1.26

Fitting 25 folds for each of 50 candidates, totalling 1250 fits
Fitting 25 folds for each of 50 candidates, totalling 1250 fits
Fitting 25 folds for each of 50 candidates, totalling 1250 fits
Best validation model: 4
Train dates: 2008-02-03 - 2015-03-17
Train MAE: 1.38
Train MSE: 3.49
Train MedAE: 1.01


KeyboardInterrupt: 

In [ ]:
print('Mean train MAE:', train_maes.mean().round(2), 'ºC')
print('Mean validation MAE:', valid_maes.mean().round(2), 'ºC')
print()
print('Mean train MSE:', train_mse.mean().round(2), 'ºC')
print('Mean validation MSE:', valid_mse.mean().round(2), 'ºC')
print()
print('Mean train MedAE:', train_medae.mean().round(2), 'ºC')
print('Mean validation MedAE:', valid_medae.mean().round(2), 'ºC')

In [21]:
# Create TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=10, test_size=7)



param_dist_gb = {
    "max_depth": [3, 5, 10, 15],
    "min_samples_split": np.arange(2, 20),
    "min_samples_leaf": np.arange(1, 11),
    "criterion": ["friedman_mse", "squared_error"],
    "n_estimators": np.arange(90,200),
    "max_features": ["sqrt", "log2", 0.2, 0.4, 0.6, 0.8], 
    "learning_rate": [0.05, 0.08, 0.09, 0.099]
}

gb = GradientBoostingRegressor()

# Models to try in each cross-validation split
models = [   
    RandomizedSearchCV(gb, param_distributions=param_dist_gb,
                             return_train_score=True,
                             cv=tscv, verbose=1, n_iter=50,
                             n_jobs=-1),
]

train_maes, valid_maes, train_mse, valid_mse, train_medae, valid_medae, best_pa = custom_cv_ts(df, models, X_train, y_train, cv=tscv)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
Fitting 10 folds for each of 50 candidates, totalling 500 fits
0
Best validation model: 0
Train dates: 2008-02-03 - 2015-06-16
Train MAE: 0.47
Train MSE: 0.41
Train MedAE: 0.34
Validation dates: 2015-06-17 - 2015-06-23
Validation MAE: 1.55
Validation MSE: 3.07
Validation MedAE: 1.56

Best Parameters:  {'n_estimators': 97, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 0.4, 'max_depth': 10, 'learning_rate': 0.08, 'criterion': 'friedman_mse'}
Fitting 10 folds for each of 50 candidates, totalling 500 fits
Fitting 10 folds for each of 50 candidates, totalling 500 fits
1
Best validation model: 0
Train dates: 2008-02-03 - 2015-06-23
Train MAE: 0.72
Train MSE: 0.95
Train MedAE: 0.54
Validation dates: 2015-06-24 - 2015-06-30
Validation MAE: 0.72
Validation MSE: 0.75
Validation MedAE: 0.84

Best Parameters:  {'n_estimators': 172, 'min_samples_split': 12, 'min_samples_leaf': 10, 'max_features': 0.8, 'max_depth': 10, '

In [23]:
print('Mean train MAE:', train_maes.mean().round(2), 'ºC')
print('Mean validation MAE:', valid_maes.mean().round(2), 'ºC')
print()
print('Mean train MSE:', train_mse.mean().round(2), 'ºC')
print('Mean validation MSE:', valid_mse.mean().round(2), 'ºC')
print()
print('Mean train MedAE:', train_medae.mean().round(2), 'ºC')
print('Mean validation MedAE:', valid_medae.mean().round(2), 'ºC')
print()
print(best_pa)

Mean train MAE: 0.91 ºC
Mean validation MAE: 1.32 ºC

Mean train MSE: 1.63 ºC
Mean validation MSE: 3.13 ºC

Mean train MedAE: 0.67 ºC
Mean validation MedAE: 1.1 ºC

[{'n_estimators': 97, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 0.4, 'max_depth': 10, 'learning_rate': 0.08, 'criterion': 'friedman_mse'}
 {'n_estimators': 172, 'min_samples_split': 12, 'min_samples_leaf': 10, 'max_features': 0.8, 'max_depth': 10, 'learning_rate': 0.05, 'criterion': 'squared_error'}
 {'n_estimators': 140, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 0.4, 'max_depth': 10, 'learning_rate': 0.05, 'criterion': 'squared_error'}
 {'n_estimators': 106, 'min_samples_split': 4, 'min_samples_leaf': 10, 'max_features': 0.6, 'max_depth': 10, 'learning_rate': 0.099, 'criterion': 'squared_error'}
 {'n_estimators': 198, 'min_samples_split': 8, 'min_samples_leaf': 10, 'max_features': 0.4, 'max_depth': 5, 'learning_rate': 0.099, 'criterion': 'friedman_mse'}
 {'n_estimators': 166, 'min_

# Ideas

- Feel free to modify `custom_cv_ts()` as you wish.
- Try different sets of features. Which are the most relevant in this problem?
- Change the target to predict more days into the future.
- Use more training data for the first split.
- Use a maximum of training days instead of always using the whole previous data, so the training window moves in time.
- Predict 1, 2 and 3 days into the future:
    - Using 3 different models that, with the actual data, predict the temperature in 1, 2 and 3 days.
	- Using a single model that predicts one day ahead and, iteratively, uses this prediction to predict the next days.


In [ ]:
# Do it before